# LightGBM regressor (time series aware)
Load splits, run baseline LightGBM, tune with TimeSeriesSplit, compare to baseline, and save artifacts under `artifacts/lightgbm`. Early stopping is applied on the validation split after selecting the best params.

Load splits from artifacts/data, run a strong LightGBM baseline, tune with TimeSeriesSplit, compare on val/test, and save outputs under `artifacts/lightgbm`.

In [12]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import joblib
from lightgbm import LGBMRegressor, early_stopping
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
from sklearn.base import clone
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
# paths to data and artifacts
_start = Path.cwd().resolve()
_candidates = [_start] + list(_start.parents)
_repo = None
for p in _candidates:
    if (p / 'notebooks/model_evaluation_final/artifacts/data/split_config.json').exists():
        _repo = p
        break
    if (p / 'data/master_dataset.parquet').exists():
        _repo = p
        break
REPO_ROOT = _repo if _repo else _start
ARTIFACTS_DIR = REPO_ROOT / 'notebooks/model_evaluation_final/artifacts'
DATA_DIR = ARTIFACTS_DIR / 'data'
MODEL_DIR = ARTIFACTS_DIR / 'lightgbm'
for d in [ARTIFACTS_DIR, DATA_DIR, MODEL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DATA_DIR, MODEL_DIR

(PosixPath('/Users/aayushrijal/Documents/GitHub/volatility_forecast/notebooks/model_evaluation_final/artifacts/data'),
 PosixPath('/Users/aayushrijal/Documents/GitHub/volatility_forecast/notebooks/model_evaluation_final/artifacts/lightgbm'))

In [14]:
# load artifacts
with open(DATA_DIR / 'feature_columns.json') as f:
    feature_cols = json.load(f)
with open(DATA_DIR / 'split_config.json') as f:
    CONFIG = json.load(f)
target_col = CONFIG['target_col']
date_col = CONFIG['date_col']

X_train = pd.read_parquet(DATA_DIR / 'X_train.parquet')
X_val = pd.read_parquet(DATA_DIR / 'X_val.parquet')
X_test = pd.read_parquet(DATA_DIR / 'X_test.parquet') if (DATA_DIR / 'X_test.parquet').exists() else pd.DataFrame(columns=feature_cols)
y_train = pd.read_parquet(DATA_DIR / 'y_train.parquet')[target_col].values
y_val = pd.read_parquet(DATA_DIR / 'y_val.parquet')[target_col].values
y_test = pd.read_parquet(DATA_DIR / 'y_test.parquet')[target_col].values if (DATA_DIR / 'y_test.parquet').exists() else np.array([])

len(X_train), len(X_val), len(X_test), len(feature_cols)

(3154, 788, 30, 20)

In [15]:
# keep raw features (LightGBM handles scaling internally)
X_train_raw = X_train.copy()
X_val_raw = X_val.copy()
X_test_raw = X_test.copy()
has_test = len(X_test_raw) > 0

len(X_train_raw), len(X_val_raw), len(X_test_raw)

(3154, 788, 30)

In [16]:
# helper functions
def metrics(y_true, y_pred):
    return {
        'rmse': float(np.sqrt(mean_squared_error(y_true, y_pred))),
        'mae': float(mean_absolute_error(y_true, y_pred)),
        'r2': float(r2_score(y_true, y_pred))
    }

def eval_model(model, name, X_tr, y_tr, X_v, y_v, X_te=None, y_te=None):
    model.fit(X_tr, y_tr)
    pred_tr = model.predict(X_tr)
    pred_v = model.predict(X_v)
    out = {
        'name': name,
        'train': metrics(y_tr, pred_tr),
        'val': metrics(y_v, pred_v)
    }
    preds = {'train': pred_tr, 'val': pred_v, 'test': None}
    if X_te is not None and len(X_te):
        pred_te = model.predict(X_te)
        out['test'] = metrics(y_te, pred_te)
        preds['test'] = pred_te
    return out, model, preds

def ts_cv_metrics(model, X, y, cv):
    fold_metrics = []
    for tr_idx, va_idx in cv.split(X):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]
        m = clone(model)
        m.fit(X_tr, y_tr)
        preds = m.predict(X_va)
        fold_metrics.append(metrics(y_va, preds))
    agg = {}
    for key in fold_metrics[0].keys():
        vals = [fm[key] for fm in fold_metrics]
        agg[key] = {'mean': float(np.mean(vals)), 'std': float(np.std(vals))}
    return agg

## Baseline LightGBM

In [17]:
baseline_lgbm = LGBMRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=64,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    min_child_samples=40,
    reg_alpha=0.0,
    reg_lambda=1.0,
    objective='regression',
    metric='rmse',
    random_state=42,
    n_jobs=-1,
    verbosity=-1
)
baseline_lgbm_out, baseline_lgbm_model, baseline_lgbm_preds = eval_model(
    baseline_lgbm,
    'lgbm_baseline',
    X_train_raw, y_train,
    X_val_raw, y_val,
    X_test_raw if has_test else None,
    y_test if has_test else None
)
baseline_lgbm_out

{'name': 'lgbm_baseline',
 'train': {'rmse': 0.005654080210083856,
  'mae': 0.003756746870337472,
  'r2': 0.8602493278779011},
 'val': {'rmse': 0.010053311467296664,
  'mae': 0.006379503254889932,
  'r2': 0.3293876967509569},
 'test': {'rmse': 0.004974819568001419,
  'mae': 0.0044455292236663295,
  'r2': -0.8264780353614403}}

## Hyperparameter tuning (TimeSeriesSplit)

In [21]:
tscv = TimeSeriesSplit(n_splits=3)

param_grid = {
    'n_estimators': [300, 600],
    'learning_rate': [0.03, 0.05],
    'max_depth': [4, 7],
    'num_leaves': [31, 63],
    'subsample': [0.7, 0.85],
    'colsample_bytree': [0.7, 0.95],
    'reg_alpha': [0.0, 0.1],
    'reg_lambda': [1.0, 2.0],
    'min_child_samples': [20, 40]
}

grid = list(ParameterGrid(param_grid))
print(f"Hyperparameter configs: {len(grid)}")

cv_records = []
best_entry = None

for params in grid:
    candidate = LGBMRegressor(
        **params,
        objective='regression',
        metric='rmse',
        random_state=42,
        n_jobs=-1,
        subsample_freq=1,
        verbosity=-1
    )

    cv_stats = ts_cv_metrics(candidate, X_train_raw, y_train, tscv)
    fitted = clone(candidate)
    fitted.fit(X_train_raw, y_train)
    entry = {
        'params': params,
        'cv': cv_stats,
        'val_rmse_mean': cv_stats['rmse']['mean'],
        'val_rmse_std': cv_stats['rmse']['std']
    }

    cv_records.append(entry)


    if (best_entry is None) or (entry['val_rmse_mean'] < best_entry['val_rmse_mean']):
        best_entry = {**entry, 'model': fitted}


cv_results_df = pd.DataFrame([{**{k: v for k, v in rec.items() if k != 'cv'}, 'cv_rmse_mean': rec['cv']['rmse']['mean'], 'cv_rmse_std': rec['cv']['rmse']['std']} for rec in cv_records])
cv_results_df_serializable = cv_results_df.copy()
cv_results_df_serializable['params'] = cv_results_df_serializable['params'].apply(json.dumps)


best_params = best_entry['params']
best_cv = best_entry['cv']

# refit best with early stopping on validation (LightGBM callback API)

best_model = LGBMRegressor(
    **best_params,
    objective='regression',
    metric='rmse',
    random_state=42,
    n_jobs=-1,
    subsample_freq=1,
    verbosity=-1
 )


best_model.fit(
    X_train_raw,
    y_train,
    eval_set=[(X_val_raw, y_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(50, verbose=False)]
 )


best_model

Hyperparameter configs: 512


LGBMRegressor(colsample_bytree=0.7, learning_rate=0.03, max_depth=4,
              metric='rmse', min_child_samples=40, n_estimators=300, n_jobs=-1,
              objective='regression', random_state=42, reg_alpha=0.1,
              reg_lambda=2.0, subsample=0.7, subsample_freq=1, verbosity=-1)

## Evaluate tuned model and select best

In [22]:
best_iteration = best_model.best_iteration_ if best_model.best_iteration_ is not None else best_model.get_params().get('n_estimators')
best_pred_train = best_model.predict(X_train_raw, num_iteration=best_iteration)
best_pred_val = best_model.predict(X_val_raw, num_iteration=best_iteration)
best_pred_test = best_model.predict(X_test_raw, num_iteration=best_iteration) if has_test else None

best_out = {
    'name': 'lgbm_tuned',
    'train': metrics(y_train, best_pred_train),
    'val': metrics(y_val, best_pred_val)
}
best_preds = {'train': best_pred_train, 'val': best_pred_val, 'test': best_pred_test}
if has_test:
    best_out['test'] = metrics(y_test, best_pred_test)

# choose best overall vs baseline LightGBM
baseline_val_rmse = baseline_lgbm_out['val']['rmse']
tuned_val_rmse = best_out['val']['rmse']
best_overall_name = 'lgbm_tuned' if tuned_val_rmse <= baseline_val_rmse else 'lgbm_baseline'
if best_overall_name == 'lgbm_tuned':
    best_overall_model = best_model
    best_overall_preds = best_preds
    best_overall_metrics = best_out
    best_overall_params = best_params
    best_overall_cv = best_cv
else:
    best_overall_model = baseline_lgbm_model
    best_overall_preds = baseline_lgbm_preds
    best_overall_metrics = baseline_lgbm_out
    best_overall_params = baseline_lgbm_model.get_params()
    best_overall_cv = None

print(f"Best model selected: {best_overall_name} (by val RMSE)")
best_overall_metrics

Best model selected: lgbm_tuned (by val RMSE)


{'name': 'lgbm_tuned',
 'train': {'rmse': 0.009036037548994211,
  'mae': 0.0056933769244386125,
  'r2': 0.6430675534761805},
 'val': {'rmse': 0.009286497847619708,
  'mae': 0.005615033211137025,
  'r2': 0.4277877303640396},
 'test': {'rmse': 0.004717498467452408,
  'mae': 0.003922820627835008,
  'r2': -0.6424165838684288}}

## Compare baseline vs tuned

In [23]:
rows = []
for label, res in [('lgbm_base', baseline_lgbm_out), ('lgbm_tuned', best_out)]:
    row = {
        'model': label,
        'val_rmse': res['val']['rmse'],
        'val_mae': res['val']['mae'],
        'val_r2': res['val']['r2']
    }
    if has_test and 'test' in res:
        row['test_rmse'] = res['test']['rmse']
        row['test_mae'] = res['test']['mae']
        row['test_r2'] = res['test']['r2']
    rows.append(row)
comparison_df = pd.DataFrame(rows).set_index('model')
print('Validation/Test comparison:')
display(comparison_df)

if 'lgbm_base' in comparison_df.index and 'lgbm_tuned' in comparison_df.index:
    diff_df = comparison_df.loc['lgbm_tuned'] - comparison_df.loc['lgbm_base']
    print('Tuned minus baseline (negative rmse/mae is better):')
    display(diff_df)
else:
    print('Cannot compute diff; missing one of lgbm_base or lgbm_tuned.')

Validation/Test comparison:


,val_rmse,val_mae,val_r2,test_rmse,test_mae,test_r2
model,,,,,,
lgbm_base,0.010053,0.006380,0.329388,0.004975,0.004446,-0.826478
lgbm_tuned,0.009286,0.005615,0.427788,0.004717,0.003923,-0.642417


Tuned minus baseline (negative rmse/mae is better):


val_rmse    -0.000767
val_mae     -0.000764
val_r2       0.098400
test_rmse   -0.000257
test_mae    -0.000523
test_r2      0.184061
dtype: float64

## Feature importance

In [24]:
importance_df = None
if hasattr(best_overall_model, 'booster_'):
    booster = best_overall_model.booster_
    importances = booster.feature_importance(importance_type='gain')
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importances
    }).sort_values(by='importance', ascending=False).reset_index(drop=True)
elif hasattr(best_overall_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_overall_model.feature_importances_
    }).sort_values(by='importance', ascending=False).reset_index(drop=True)
importance_df.head(15) if importance_df is not None else None

,feature,importance
0,vix,1.463722
1,rv_vix_spread_20d,1.278835
2,vix_term,0.475621
3,spy_ret_10d,0.442031
4,drawdown_60d,0.440274
5,spy_vol_10d,0.381598
6,spy_vol_5d,0.233590
7,spy_ret_20d,0.195141
8,corr_spy_hyg_60d,0.114420
9,corr_spy_hyg_20d,0.079581


## Save artifacts

In [26]:
print(f"Saving best model: {best_overall_name}")
print("Hyperparameters:")
print(json.dumps(best_overall_params, indent=2))

metrics_payload = {
    'base': baseline_lgbm_out,
    'tuned': best_out,
    'best': best_overall_metrics,
    'best_model': best_overall_name,
    'best_cv': best_overall_cv
}
hyperparams_payload = {
    'best_model_name': best_overall_name,
    'best_params': best_overall_params
}

with open(MODEL_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics_payload, f, indent=2)
with open(MODEL_DIR / 'hyperparams.json', 'w') as f:
    json.dump(hyperparams_payload, f, indent=2)

if importance_df is not None:
    importance_df.to_parquet(MODEL_DIR / 'feature_importance.parquet', index=False)
    importance_df.to_csv(MODEL_DIR / 'feature_importance.csv', index=False)

joblib.dump(best_overall_model, MODEL_DIR / 'model.joblib')

pred_frames = [
    pd.DataFrame({'split': 'train', 'y_true': y_train, 'y_pred': best_overall_preds['train']}),
    pd.DataFrame({'split': 'val', 'y_true': y_val, 'y_pred': best_overall_preds['val']})
]
if has_test and best_overall_preds['test'] is not None:
    pred_frames.append(pd.DataFrame({'split': 'test', 'y_true': y_test, 'y_pred': best_overall_preds['test']}))
pd.concat(pred_frames, ignore_index=True).to_parquet(MODEL_DIR / 'preds.parquet', index=False)

cv_results_df_serializable.to_parquet(MODEL_DIR / 'cv_results.parquet', index=False)

best_overall_params

Saving best model: lgbm_tuned
Hyperparameters:
{
  "colsample_bytree": 0.7,
  "learning_rate": 0.03,
  "max_depth": 4,
  "min_child_samples": 40,
  "n_estimators": 300,
  "num_leaves": 31,
  "reg_alpha": 0.1,
  "reg_lambda": 2.0,
  "subsample": 0.7
}


{'colsample_bytree': 0.7,
 'learning_rate': 0.03,
 'max_depth': 4,
 'min_child_samples': 40,
 'n_estimators': 300,
 'num_leaves': 31,
 'reg_alpha': 0.1,
 'reg_lambda': 2.0,
 'subsample': 0.7}